In [2]:
%pip install transformers


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [5]:
%pip install kenlm
%pip install pyctcdecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 425.0/425.0 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kenlm: filename=kenlm-0.1-cp38-cp38-linux_x86_64.whl size=2991627 sha256=d8c07365eb87c9e438deff8d370ddd87aa2b1c87bf007b585a93f82f8cc5bd18
  Stored in directory: /root/.cache/pip/wheels/d4/3c/44/389ed64995f0e7382ab7c38ef13e57fb1a5e5e111350ce6807
Successfully built kenlm

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 414.9/414.9 kB 26.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
from transformers import pipeline
import torch
import logging

MODEL_NAME = "wannaphong/wav2vec2-large-xlsr-53-th-cv8-newmm"  # specify the model name
lang = "th"  # change to Thai langauge

device = "cuda:6" if torch.cuda.is_available() else "CPU"

logging.getLogger("pytorch_transformers.tokenization_utils").setLevel(logging.ERROR)


pipe = pipeline(
    task="automatic-speech-recognition",
    model=MODEL_NAME,
    chunk_length_s=30,
    device=device,
)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
import pandas as pd

def match_and_add_value(filename, match_column, match_value, target_column, new_value):
    try:
        df = pd.read_csv(filename,encoding= 'unicode_escape')
        df.loc[df[match_column] == match_value, target_column] = new_value
        df.to_csv(filename, index=False)
    except Exception as e:
        print(f"An error occurred: {str(e)}")

In [ ]:
import os

folder_path = 'modified_wave'

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    
    text = pipe(file_path)["text"] # give audio mp3 and transcribe text
    match_and_add_value("transcribe.csv", "path", filename, "transcribe", text)


In [9]:
def run(file_path,filename,df):
    text = pipe(file_path)["text"] # give audio mp3 and transcribe text
    match_and_add_value("transcribe.csv", "path", filename, "transcribe", text,df)

In [2]:
def get_text (filename,folder_path):
    file_path = os.path.join(folder_path, filename)
    text = pipe(file_path)["text"]
    return [filename,text]

In [ ]:
import os
import concurrent.futures
import warnings
from threading import Lock

warnings.filterwarnings("ignore")

def progress_indicator(future):
    global lock, tasks_total, tasks_completed
    # obtain the lock
    with lock:
        # update the counter
        tasks_completed += 1
        # report progress
        if tasks_completed % 1000 == 0:
            print(f'{tasks_completed}/{tasks_total} completed, {tasks_total-tasks_completed} remain.')

percentage = 0

queue = []
folder_path = 'modified_wave'
# create a lock for the counter
lock = Lock()
# total tasks we will execute
tasks_total = 49637
# total completed tasks
tasks_completed = 0

with concurrent.futures.ThreadPoolExecutor(max_workers=6) as executor:
    future_to_text = {executor.submit(get_text,filename,folder_path): filename for filename in os.listdir(folder_path)} 
    for future in concurrent.futures.as_completed(future_to_text):
        queue.append(future.result())
        future.add_done_callback(progress_indicator)

with open("output_2.txt", "w") as txt_file:
    for line in queue:
        txt_file.write(" ".join(line) + "\n")

In [35]:
import os
import pandas as pd
import numpy as np
import concurrent.futures

df = pd.read_csv("transcribe.csv")

folder_path = 'modified_wave'

with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
    # Submit tasks to the thread pool
    for filename in os.listdir(folder_path):
        indexes = df.loc[df['path'] == filename].index
        print(indexes)
        if df.at[indexes, 'transcribe'] is np.nan:
            file_path = os.path.join(folder_path, filename)
            future = executor.submit(pipe, file_path)


Int64Index([], dtype='int64')


InvalidIndexError: Int64Index([], dtype='int64')

In [ ]:
print(queue)

In [21]:
if len(queue) != 0:
    print(queue.pop(0))
else:
    print('Queue is empty')

Queue is empty


In [ ]:
import pandas as pd
c_queue = queue
while len(c_queue) != 0:
    result = c_queue.pop(0)
    df = pd.read_csv("transcribe2.csv")
    df.loc[df['path'] == result[0], 'transcribe'] = result[1]
df.to_csv("transcribe2_trans.csv", index=False)